# Used Car Classifcication
**ML Project 2 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonnell

Descriptions: The first part should be a description which models/classifiers/regressors achieved the best 
    results with your datasets datasets, and what you did to improve those results along the way.

    - Through research we found that a model called RandomForestClassifier worked the best after testing a couple different model including
    Niave Bayes and DecisionTreeClassifier. We improved our results by using a feauture engineer and found that selecting all of the features
    proved to provide the best results. This was different from our oroginal assumptions that the type of car would be mostly based upon the price,
    size, and fuel of the car.



### Step 1
Import the necessary packages and frameworks.

In [1]:
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from seaborn import load_dataset

from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

import category_encoders as ce

ModuleNotFoundError: No module named 'pyspark'

### Step 2
Read in .csv files and save as DataFrames.

In [ ]:
data_train_df = pd.read_csv("cars_class_train.csv")#, nrows=86108) 
data_test_df = pd.read_csv("cars_class_test.csv")#, nrows=86108)
data_train_ft = data_train_df.drop('type', axis=1)
data_train_tgt = data_train_df["type"]
data_train_tgt_copy = data_train_df["type"]

### Step 3
Look at the types of data and get a quick look at what each data type looks like. From there we can select the features we want to try and use. Original assumption
was that the type of car would be mostly based upon the price, size, and fuel of the car.


In [ ]:
# To see what kind of data we're working with
print(data_test_df.dtypes)
print(data_train_ft.dtypes)
print(data_train_ft.head(5))

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
Unnamed: 0        int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
pa

### Step 5
Convert data from catgeorical to numerical using a binary encoder. From there we want to get the best features for the data to analyze so we use the SelectKBest and the more features we have, the more accurate
the model seems to be from testing.

In [ ]:
features = ['url',
           'region',
           'region_url',
           'price',
           'year',
           'manufacturer',
           'model',
           'condition',
           'cylinders',
           'fuel',
           'odometer',
           'title_status',
           'transmission',
           'VIN',
           'drive',
           'size',
           'paint_color',
           'image_url',
           'description',
           'state',
           'lat',
           'long',
           'posting_date'
          ]


data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'], utc=True)
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'])
data_train_ft['posting_date'] = (pd.DatetimeIndex(data_train_ft['posting_date']).day) + (pd.DatetimeIndex(data_train_ft['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_train_ft['posting_date']).year)*365)
data_train_ft['posting_date'] = data_train_ft['posting_date'].astype(float)

data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'], utc=True)
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'])
data_test_df['posting_date'] = (pd.DatetimeIndex(data_test_df['posting_date']).day) + (pd.DatetimeIndex(data_test_df['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_test_df['posting_date']).year)*365)
data_test_df['posting_date'] = data_test_df['posting_date'].astype(float)
data_train_ft = data_train_ft[features]
data_train_ft = data_train_ft.fillna(0)

#data_test_ft = data_test_df
data_test_df = data_test_df[features]
data_test_df = data_test_df.fillna(0)

LE_encoder = OrdinalEncoder(features)
data_train_ft = LE_encoder.fit_transform(data_train_ft)
data_test_df = LE_encoder.transform(data_test_df)
'''
#print(f'{r}')
def target_encode_multiclass(train, test, train_tgt): #X,y are pandas df and series
    train_tgt=train_tgt.astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(train_tgt)
    y_onehot=enc.transform(train_tgt)
    class_names=y_onehot.columns  #names of onehot encoded columns
    train_obj=train.select_dtypes('object') #separate categorical columns
    train=train.select_dtypes(exclude='object') 
    test_obj=test.select_dtypes('object') #separate categorical columns
    test=test.select_dtypes(exclude='object') 
    for class_ in class_names:
        enc=ce.TargetEncoder()
        enc.fit(train_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(train_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        train=pd.concat([train,temp],axis=1)    #add to original dataset
        r = test_obj.shape[0]
        c = y_onehot[class_].shape[0]
        print(f'{r}, {c}')
        enc.fit(test_obj,y_onehot[class_].iloc[0:(test_obj.shape[0])]) #convert all categorical 
        temp=enc.transform(test_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        test=pd.concat([test,temp],axis=1)    #add to original dataset
    return train, test
data_train_ft, data_test_df = target_encode_multiclass(data_train_ft, data_test_df, data_train_tgt)
'''
r, c = data_train_df.shape
print(f'{r}, {c}')
r = data_train_tgt.shape[0]
print(f'{r}')
r, c = data_test_df.shape
print(f'{r}, {c}')
print('Train ft')
print(data_train_ft.dtypes)
print('\n Test df')
print(data_test_df.dtypes)
print('\n Train tgt')
print(data_train_tgt.dtypes)
print('\n Train Ft')
print(data_train_ft.head(5))
print('\n Test df')
print(data_test_df.head(5))
print('\n Train Tgt')
print(data_train_tgt.head(5))

data_train_ft.head()
data_test_df.head()


200916, 25
200916
86108, 23
Train ft
url               int64
region            int64
region_url        int64
price             int64
year            float64
manufacturer      int64
model             int64
condition         int64
cylinders         int64
fuel              int64
odometer        float64
title_status      int64
transmission      int64
VIN               int64
drive             int64
size              int64
paint_color       int64
image_url         int64
description       int64
state             int64
lat             float64
long            float64
posting_date    float64
dtype: object

 Test df
url             float64
region            int64
region_url        int64
price             int64
year            float64
manufacturer      int64
model           float64
condition         int64
cylinders         int64
fuel              int64
odometer        float64
title_status      int64
transmission      int64
VIN             float64
drive             int64
size              int64
pai

,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,...,VIN,drive,size,paint_color,image_url,description,state,lat,long,posting_date
0,-1.0000,94,94,3500,"2,013.0000",36,"1,765.0000",4,2,1,...,2.0000,1,4,9,-1.0000,-1.0000,10,29.4589,-98.5699,883.0000
1,-1.0000,263,268,8500,"2,004.0000",11,838.0000,5,4,1,...,2.0000,1,2,3,-1.0000,-1.0000,7,42.0990,-75.9180,891.0000
2,-1.0000,83,83,14997,"2,011.0000",33,944.0000,5,4,1,...,-1.0000,4,1,4,-1.0000,-1.0000,3,38.6050,-121.4019,883.0000
3,-1.0000,328,369,5450,"2,014.0000",2,136.0000,4,2,1,...,2.0000,3,1,9,-1.0000,-1.0000,5,42.2421,-84.4282,884.0000
4,-1.0000,101,101,4500,"2,010.0000",4,181.0000,1,1,1,...,2.0000,3,1,1,-1.0000,-1.0000,15,27.4547,-82.5359,876.0000


### Step 6
Feauture Engineering

In [ ]:
ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, 
                                  k=23)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

print(np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()])
print(data_train_ft.columns.value_counts().count())
new_features = np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()]
data_train_ft = data_train_ft[new_features]
data_test_df = data_test_df[new_features]

['url' 'transmission' 'long' 'lat' 'state' 'description' 'image_url'
 'paint_color' 'size' 'drive' 'VIN' 'title_status' 'region' 'odometer'
 'fuel' 'cylinders' 'condition' 'model' 'manufacturer' 'year' 'price'
 'region_url' 'posting_date']
23


### Step 6
Look at data before changing all data types to floats, convert all data type to the same type (float) and look at the updated table.

In [ ]:
print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft = data_train_ft.astype(float)
data_test_df = data_test_df.astype(float)

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft:


,url,transmission,long,lat,state,description,image_url,paint_color,size,drive,...,odometer,fuel,cylinders,condition,model,manufacturer,year,price,region_url,posting_date
0,1,1,-93.5701,41.6002,1,1,1,1,1,1,...,"155,815.0000",1,1,1,1,1,"2,011.0000",8995,1,880.0000
1,2,1,-79.5424,40.3074,2,2,2,2,2,2,...,"72,953.0000",1,1,2,2,2,"1,954.0000",4500,2,892.0000
2,3,1,-120.6676,35.2468,3,3,3,3,3,2,...,0.0000,1,2,1,3,3,"2,013.0000",16495,3,876.0000
3,4,1,-78.6434,35.7489,4,4,4,4,3,3,...,"64,855.0000",1,1,3,4,4,"2,014.0000",14900,4,887.0000
4,5,2,-82.4510,43.0172,5,5,5,5,1,4,...,"10,873.0000",1,1,4,5,5,"2,019.0000",35990,5,884.0000
5,6,1,-76.9991,39.8264,2,6,6,5,1,1,...,"97,465.0000",1,2,5,6,3,"2,012.0000",12900,6,872.0000
6,7,1,-83.6511,41.3900,6,7,7,6,1,3,...,"90,360.0000",1,2,4,7,2,"2,014.0000",6999,7,889.0000
7,8,1,-74.2817,40.4688,2,8,8,7,2,2,...,"84,317.0000",2,1,4,8,6,"2,011.0000",34990,8,873.0000
8,9,2,-73.9679,40.5829,7,9,9,2,2,3,...,"36,400.0000",1,1,3,9,7,"2,012.0000",8975,9,875.0000
9,10,1,-71.3347,41.7262,8,10,10,8,2,3,...,"117,142.0000",1,2,4,10,8,"2,008.0000",4000,10,883.0000


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   url           200916 non-null  int64  
 1   transmission  200916 non-null  int64  
 2   long          200916 non-null  float64
 3   lat           200916 non-null  float64
 4   state         200916 non-null  int64  
 5   description   200916 non-null  int64  
 6   image_url     200916 non-null  int64  
 7   paint_color   200916 non-null  int64  
 8   size          200916 non-null  int64  
 9   drive         200916 non-null  int64  
 10  VIN           200916 non-null  int64  
 11  title_status  200916 non-null  int64  
 12  region        200916 non-null  int64  
 13  odometer      200916 non-null  float64
 14  fuel          200916 non-null  int64  
 15  cylinders     200916 non-null  int64  
 16  condition     200916 non-null  int64  
 17  model         200916 non-n

None

data_train_ft:


,url,transmission,long,lat,state,description,image_url,paint_color,size,drive,...,odometer,fuel,cylinders,condition,model,manufacturer,year,price,region_url,posting_date
0,1.0000,1.0000,-93.5701,41.6002,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,"155,815.0000",1.0000,1.0000,1.0000,1.0000,1.0000,"2,011.0000","8,995.0000",1.0000,880.0000
1,2.0000,1.0000,-79.5424,40.3074,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,...,"72,953.0000",1.0000,1.0000,2.0000,2.0000,2.0000,"1,954.0000","4,500.0000",2.0000,892.0000
2,3.0000,1.0000,-120.6676,35.2468,3.0000,3.0000,3.0000,3.0000,3.0000,2.0000,...,0.0000,1.0000,2.0000,1.0000,3.0000,3.0000,"2,013.0000","16,495.0000",3.0000,876.0000
3,4.0000,1.0000,-78.6434,35.7489,4.0000,4.0000,4.0000,4.0000,3.0000,3.0000,...,"64,855.0000",1.0000,1.0000,3.0000,4.0000,4.0000,"2,014.0000","14,900.0000",4.0000,887.0000
4,5.0000,2.0000,-82.4510,43.0172,5.0000,5.0000,5.0000,5.0000,1.0000,4.0000,...,"10,873.0000",1.0000,1.0000,4.0000,5.0000,5.0000,"2,019.0000","35,990.0000",5.0000,884.0000
5,6.0000,1.0000,-76.9991,39.8264,2.0000,6.0000,6.0000,5.0000,1.0000,1.0000,...,"97,465.0000",1.0000,2.0000,5.0000,6.0000,3.0000,"2,012.0000","12,900.0000",6.0000,872.0000
6,7.0000,1.0000,-83.6511,41.3900,6.0000,7.0000,7.0000,6.0000,1.0000,3.0000,...,"90,360.0000",1.0000,2.0000,4.0000,7.0000,2.0000,"2,014.0000","6,999.0000",7.0000,889.0000
7,8.0000,1.0000,-74.2817,40.4688,2.0000,8.0000,8.0000,7.0000,2.0000,2.0000,...,"84,317.0000",2.0000,1.0000,4.0000,8.0000,6.0000,"2,011.0000","34,990.0000",8.0000,873.0000
8,9.0000,2.0000,-73.9679,40.5829,7.0000,9.0000,9.0000,2.0000,2.0000,3.0000,...,"36,400.0000",1.0000,1.0000,3.0000,9.0000,7.0000,"2,012.0000","8,975.0000",9.0000,875.0000
9,10.0000,1.0000,-71.3347,41.7262,8.0000,10.0000,10.0000,8.0000,2.0000,3.0000,...,"117,142.0000",1.0000,2.0000,4.0000,10.0000,8.0000,"2,008.0000","4,000.0000",10.0000,883.0000


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   url           200916 non-null  float64
 1   transmission  200916 non-null  float64
 2   long          200916 non-null  float64
 3   lat           200916 non-null  float64
 4   state         200916 non-null  float64
 5   description   200916 non-null  float64
 6   image_url     200916 non-null  float64
 7   paint_color   200916 non-null  float64
 8   size          200916 non-null  float64
 9   drive         200916 non-null  float64
 10  VIN           200916 non-null  float64
 11  title_status  200916 non-null  float64
 12  region        200916 non-null  float64
 13  odometer      200916 non-null  float64
 14  fuel          200916 non-null  float64
 15  cylinders     200916 non-null  float64
 16  condition     200916 non-null  float64
 17  model         200916 non-n

None

### Step 7
Scale the data to get rid of any outliers that would affect the model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scale_datasets(x_train, x_test):

  """
  Standard Scale test and train data
  """
  standard_scaler = MinMaxScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

data_train_ft, data_test_df = scale_datasets(data_train_ft, data_test_df)

### Step 8
Create a library of different models to train and track the accuracy of later. Like the feature selection, having a wide variety of models to test out helps us ensure that our predictions are as accurate as possible.

In [ ]:
scaler = skpre.StandardScaler()

pipelines_to_try = {}

# tested KNN, but was not accurate

for k in range(36,38,2):
    pipelines_to_try[f'DTC-{k}'] =  tree.DecisionTreeClassifier(max_depth=k)
for k in range(100,120, 30):
    pipelines_to_try[f'RandomForestClassifier-{k}'] = RandomForestClassifier(n_estimators=k)
baseline = dummy.DummyClassifier(strategy="uniform")
pipelines_to_try1 = {}
# scale the data
for model_name in pipelines_to_try:
    pipelines_to_try1[f'std_{model_name}_pipe'] = pipeline.make_pipeline(scaler, 
                                                      pipelines_to_try[model_name])

### Step 9
Use cross-validation to test the accuracy of the models.

In [ ]:
accuracy_scores = {}
loo = skms.LeaveOneOut()
for name, model in pipelines_to_try1.items():
    loo = skms.LeaveOneOut()
    print(f'Currently modeling using : {name}')
    scores = skms.cross_val_score(model,
                                  data_train_ft,
                                  data_train_tgt,
                                  #cv=loo,
                                  #n_jobs=-1,
                                  cv=10,
                                  scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[name] = mean_accuracy
    print(f'{name}: {mean_accuracy:.3f}')
# Took 14m 36.9s on my mac

Currently modeling using : std_DTC-36_pipe
std_DTC-36_pipe: 0.749
Currently modeling using : std_RandomForestClassifier-100_pipe
std_RandomForestClassifier-100_pipe: 0.801


### Step 10
Find the pipeline with the highest accuracy and save it for testing.

In [ ]:
best_pipeline_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest pipeline: {best_pipeline_name} (accuracy = {accuracy_scores[best_pipeline_name]:.3f})')
final_pipeline = pipelines_to_try1[best_pipeline_name]

# Best model seems to be RandomForestClassifier (accuracy = 0.831)


Best pipeline: std_RandomForestClassifier-100_pipe (accuracy = 0.801)


### Step 11
Fit and test the highest-scoring model against the testing data. Save the results in submission.csv for uploading to Kaggle.

In [ ]:
fit = final_pipeline.fit(data_train_ft, data_train_tgt)
predictions = fit.predict(data_test_df)
make_submission_df = pd.read_csv("cars_class_test.csv")
print(predictions)
# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')
# write to csv file
make_submission_df['type'] = predictions
make_submission_df.to_csv('submission.csv',sep=',')
# display message
print("Saved predictions to csv file.")

['sedan' 'SUV' 'SUV' ... 'SUV' 'truck' 'other']
Saved predictions to csv file.
